<a href="https://colab.research.google.com/github/cloud-annotations/google-colab-training/blob/master/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
The only thing you **NEED** to change is `ANNOTATIONS_FOLDER`.

You can easily create your own annotations for free at [Cloud Annotations](https://cloud.annotations.ai).

### Steps
1. Start a new project
1. Choose **`localization`** as the project type
1. Label your images and videos
1. Export an annotation folder by clicking `File > Export as Create ML`
1. Unzip the folder and upload it to Google Drive


In [0]:
################################################################################
# Things to change:
ANNOTATIONS_FOLDER = 'THE_NAME_OF_YOUR_FOLDER_IN_GOOGLE_DRIVE'
NUM_TRAIN_STEPS = 500
################################################################################

import os
GOOGLE_DRIVE_MOUNT    = '/content/drive'
ANNOTATIONS_PATH      = os.path.join(GOOGLE_DRIVE_MOUNT, 'My Drive', ANNOTATIONS_FOLDER)
ANNOTATIONS_JSON_PATH = os.path.join(ANNOTATIONS_PATH, 'annotations.json')

CHECKPOINT_PATH = '/content/checkpoint'
OUTPUT_PATH     = '/content/output'
EXPORTED_PATH   = '/content/exported'
DATA_PATH       = '/content/data'

LABEL_MAP_PATH    = os.path.join(DATA_PATH, 'label_map.pbtxt')
TRAIN_RECORD_PATH = os.path.join(DATA_PATH, 'train.record')
VAL_RECORD_PATH   = os.path.join(DATA_PATH, 'val.record')

# Install the TensorFlow Object Detection API
In order to use the TensorFlow Object Detection API, we need to clone it's GitHub Repo.

### Dependencies
Most of the dependencies required come preloaded in Google Colab. The only additional package we need to install is TensorFlow.js, which is used for converting our trained model to a model that is compatible for the web.

### Protocol Buffers
The TensorFlow Object Detection API relies on what are called `protocol buffers` (also known as `protobufs`). Protobufs are a language neutral way to describe information. That means you can write a protobuf once and then compile it to be used with other languages, like Python, Java or C.

The `protoc` command used below is compiling all the protocol buffers in the `object_detection/protos` folder for Python.

### Environment
To use the object detection api we need to add it to our `PYTHONPATH` along with `slim` which contains code for training and evaluating several widely used Convolutional Neural Network (CNN) image classification models.

In [0]:
import os

!cd /content
!git clone https://github.com/tensorflow/models.git
!pip install --no-deps tensorflowjs==1.4.0

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

pwd = os.getcwd()
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'

# Test the setup
If everything was set up properly and nothing went wrong, we should be able to run this command.

In [0]:
!python object_detection/builders/model_builder_test.py

# Mount Google Drive
In order to use files from Google Drive we need to mount it to Colab.

When running this command for the first time it will ask you to open a link to accept permissions. After doing so, it will give you an authorization code that you can copy and paste below to complete the mounting process.

In [0]:
from google.colab import drive
drive.mount(GOOGLE_DRIVE_MOUNT)

# Generate a Label Map
One piece of data the Object Detection API needs is a label map protobuf. The label map associates an integer id to the text representation of the label. The ids are indexed by 1, meaning the first label will have an id of 1 not 0.

Here is an example of what a label map looks like:
```
item {
  id: 1
  name: 'Cat'
}

item {
  id: 2
  name: 'Dog'
}

item {
  id: 3
  name: 'Gold Fish'
}
```


In [0]:
import os
import json

# Get a list of labels from the annotations.json
labels = {}
with open(ANNOTATIONS_JSON_PATH) as f:
  annotations = json.load(f)
  labels = {l['label'] for a in annotations for l in a['annotations']}

# Create a file named label_map.pbtxt
os.makedirs(DATA_PATH, exist_ok=True)
with open(LABEL_MAP_PATH, 'w') as f:
  # Loop through all of the labels and write each label to the file with an id
  for idx, label in enumerate(labels):
    f.write('item {\n')
    f.write("\tname: '{}'\n".format(label))
    f.write('\tid: {}\n'.format(idx + 1)) # indexes must start at 1
    f.write('}\n')

# Generate TFRecords
The TensorFlow Object Detection API expects our data to be in the format of TFRecords.

The TFRecord format is a collection of serialized feature dicts, one for each image, looking something like this:
```
{
  'image/height': 1800,
  'image/width': 2400,
  'image/filename': 'image1.jpg',
  'image/source_id': 'image1.jpg',
  'image/encoded': ACTUAL_ENCODED_IMAGE_DATA_AS_BYTES,
  'image/format': 'jpeg',
  'image/object/bbox/xmin': [0.7255949630314233, 0.8845598428835489],
  'image/object/bbox/xmax': [0.9695875693160814, 1.0000000000000000],
  'image/object/bbox/ymin': [0.5820120073891626, 0.1829972290640394],
  'image/object/bbox/ymax': [1.0000000000000000, 0.9662484605911330],
  'image/object/class/text': (['Cat', 'Dog']),
  'image/object/class/label': ([1, 2])
}
```

In [0]:
import os
import io
import json
import random

import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def create_tf_record(annotations, label_map, image_path, output):
  # Create a train.record TFRecord file.
  with tf.python_io.TFRecordWriter(output) as writer:
    # Loop through all the training examples.
    for annotation in annotations:
      try:
        # Make sure the image is actually a file
        img_path = os.path.join(image_path, annotation['image'])   
        if not os.path.isfile(img_path):
          continue
          
        # Read in the image.
        with tf.gfile.GFile(img_path, 'rb') as fid:
          encoded_jpg = fid.read()

        # Open the image with PIL so we can check that it's a jpeg and get the image
        # dimensions.
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = PIL.Image.open(encoded_jpg_io)
        if image.format != 'JPEG':
          raise ValueError('Image format not JPEG')

        width, height = image.size

        # Initialize all the arrays.
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []

        # The class text is the label name and the class is the id. If there are 3
        # cats in the image and 1 dog, it may look something like this:
        # classes_text = ['Cat', 'Cat', 'Dog', 'Cat']
        # classes      = [  1  ,   1  ,   2  ,   1  ]

        # For each image, loop through all the annotations and append their values.
        for a in annotation['annotations']:
          coord = a['coordinates']
          xmin = coord['x'] - (coord['width'] / 2)
          xmax = xmin + coord['width']
          ymin = coord['y'] - (coord['height'] / 2)
          ymax = ymin + coord['height']
          xmins.append(max(xmin / width, 0))
          xmaxs.append(min(xmax / width, 1))
          ymins.append(max(ymin / height, 0))
          ymaxs.append(min(ymax / height, 1))
          label = a['label']
          classes_text.append(label.encode('utf8'))
          classes.append(label_map[label])
      
        # Create the TFExample.
        tf_example = tf.train.Example(features=tf.train.Features(feature={
          'image/height': dataset_util.int64_feature(height),
          'image/width': dataset_util.int64_feature(width),
          'image/filename': dataset_util.bytes_feature(annotation['image'].encode('utf8')),
          'image/source_id': dataset_util.bytes_feature(annotation['image'].encode('utf8')),
          'image/encoded': dataset_util.bytes_feature(encoded_jpg),
          'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
          'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
          'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
          'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
          'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
          'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
          'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
        if tf_example:
          # Write the TFExample to the TFRecord.
          writer.write(tf_example.SerializeToString())
      except ValueError:
        print('Invalid example, ignoring.')
        pass
      except IOError:
        # print("Can't read example, ignoring.")
        pass

with open(ANNOTATIONS_JSON_PATH) as f:
  annotations = json.load(f)
  # Load the label map we created.
  label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)

  random.seed(42)
  random.shuffle(annotations)
  num_train = int(0.7 * len(annotations))
  train_examples = annotations[:num_train]
  val_examples = annotations[num_train:]

  create_tf_record(train_examples, label_map, ANNOTATIONS_PATH, TRAIN_RECORD_PATH)
  create_tf_record(val_examples, label_map, ANNOTATIONS_PATH, VAL_RECORD_PATH)

# Download a base model
Training a model from scratch can take days and tons of data. We can mitigate this by using a pretrained model checkpoint. Instead of starting from nothing, we can add to what was already learned with our own data.

We can get a download a checkpoint from the [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).

We can download the checkpoint to our training run with the following code:

In [0]:
import os
import tarfile

import six.moves.urllib as urllib

download_base = 'http://download.tensorflow.org/models/object_detection/'
model = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz'
tmp = '/content/checkpoint.tar.gz'

if not (os.path.exists(CHECKPOINT_PATH)):
  # Download the checkpoint
  opener = urllib.request.URLopener()
  opener.retrieve(download_base + model, tmp)

  # Extract all the `model.ckpt` files.
  with tarfile.open(tmp) as tar:
    for member in tar.getmembers():
      member.name = os.path.basename(member.name)
      if 'model.ckpt' in member.name:
        tar.extract(member, path=CHECKPOINT_PATH)

  os.remove(tmp)

# Model config
The final thing we need to do is inject our pipline with the amount of labels we have and where to find the label map, TFRecord and model checkpoint. We also need to change the the batch size, because the default batch size of 128 is too large for Colab to handle.

In [0]:
import re

from google.protobuf import text_format

from object_detection.utils import config_util
from object_detection.utils import label_map_util

pipeline_skeleton = '/content/models/research/object_detection/samples/configs/ssd_mobilenet_v1_quantized_300x300_coco14_sync.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(label_map.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
  'model.{}.num_classes'.format(meta_arch): num_classes,
  'train_config.batch_size': 24,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VAL_RECORD_PATH,
  'train_config.fine_tune_checkpoint': os.path.join(CHECKPOINT_PATH, 'model.ckpt'),
  'label_map_path': LABEL_MAP_PATH
}

configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, DATA_PATH)

# Start training

In [0]:
!rm -rf $OUTPUT_PATH
!python -m object_detection.model_main \
    --pipeline_config_path=$DATA_PATH/pipeline.config \
    --model_dir=$OUTPUT_PATH \
    --num_train_steps=$NUM_TRAIN_STEPS \
    --num_eval_steps=100

# Export inference graph

In [0]:
import os
import re

regex = re.compile(r"model\.ckpt-([0-9]+)\.index")
numbers = [int(regex.search(f).group(1)) for f in os.listdir(OUTPUT_PATH) if regex.search(f)]
TRAINED_CHECKPOINT_PREFIX = os.path.join(OUTPUT_PATH, 'model.ckpt-{}'.format(max(numbers)))

!python -m object_detection.export_inference_graph \
  --pipeline_config_path=$DATA_PATH/pipeline.config \
  --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
  --output_directory=$EXPORTED_PATH

# Convert the model

In [0]:
!tensorflowjs_converter \
  --input_format=tf_frozen_model \
  --output_format=tfjs_graph_model \
  --output_node_names='Postprocessor/ExpandDims_1,Postprocessor/Slice' \
  --quantization_bytes=1 \
  --skip_op_check \
  $EXPORTED_PATH/frozen_inference_graph.pb \
  /content/model_web

import json

from object_detection.utils.label_map_util import get_label_map_dict

label_map = get_label_map_dict(LABEL_MAP_PATH)
label_array = [k for k in sorted(label_map, key=label_map.get)]

with open(os.path.join('/content/model_web', 'labels.json'), 'w') as f:
  json.dump(label_array, f)

!cd /content/model_web && zip -r /content/model_web.zip *

# Download the model

In [0]:
from google.colab import files
files.download('/content/model_web.zip') 